# 6.5.2 Create Heatmaps for Weather Parameters

In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Mersing,MY,2020-10-04 00:40:09,2.43,103.84,77.00,88,20,3.36
1,1,Svetlogorsk,BY,2020-10-04 00:40:09,52.63,29.73,56.88,73,86,13.58
2,2,Juli,PE,2020-10-04 00:40:09,-16.22,-69.45,44.67,72,0,6.96
3,3,Ushuaia,AR,2020-10-04 00:40:09,-54.80,-68.30,42.80,70,40,7.27
4,4,Kiama,AU,2020-10-04 00:40:09,-34.68,150.87,78.01,61,52,8.01


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [6]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heatmap of percent windiness
locations = city_data_df[["Lat", "Lng"]]
winds = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=winds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

# 6.5.3 Get Vacation Criteria

In [10]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [11]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Mersing,MY,2020-10-04 00:40:09,2.43,103.84,77.00,88,20,3.36
4,4,Kiama,AU,2020-10-04 00:40:09,-34.68,150.87,78.01,61,52,8.01
5,5,Vaini,TO,2020-10-04 00:40:09,-21.20,-175.20,75.20,94,90,11.41
6,6,Thinadhoo,MV,2020-10-04 00:40:09,0.53,72.93,83.07,70,100,6.42
8,8,Airai,TL,2020-10-04 00:40:09,-8.93,125.41,75.63,39,89,4.12
14,14,Haapiti,PF,2020-10-04 00:40:10,-17.57,-149.87,86.00,62,20,5.82
22,22,Biak,ID,2020-10-04 00:40:11,-0.91,122.88,82.72,68,100,7.47
23,23,Kathu,TH,2020-10-04 00:40:11,7.92,98.33,81.00,100,40,6.91
28,28,Nortelandia,BR,2020-10-04 00:40:12,-14.45,-56.80,88.56,26,67,2.24
29,29,Tezu,IN,2020-10-04 00:40:12,27.92,96.17,79.63,68,8,0.54


In [12]:
preferred_cities_df.count()

City_ID       173
City          173
Country       173
Date          173
Lat           173
Lng           173
Max Temp      173
Humidity      173
Cloudiness    173
Wind Speed    173
dtype: int64

# 6.5.4 Map Vacation Criteria

In [13]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Mersing,MY,77.00,2.43,103.84,
4,Kiama,AU,78.01,-34.68,150.87,
5,Vaini,TO,75.20,-21.20,-175.20,
6,Thinadhoo,MV,83.07,0.53,72.93,
8,Airai,TL,75.63,-8.93,125.41,
14,Haapiti,PF,86.00,-17.57,-149.87,
22,Biak,ID,82.72,-0.91,122.88,
23,Kathu,TH,81.00,7.92,98.33,
28,Nortelandia,BR,88.56,-14.45,-56.80,
29,Tezu,IN,79.63,27.92,96.17,


In [14]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

### Iterate Through hotel_df DataFrame

In [15]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")

In [16]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Mersing,MY,77.00,2.43,103.84,
4,Kiama,AU,78.01,-34.68,150.87,
5,Vaini,TO,75.20,-21.20,-175.20,
6,Thinadhoo,MV,83.07,0.53,72.93,
8,Airai,TL,75.63,-8.93,125.41,
14,Haapiti,PF,86.00,-17.57,-149.87,
22,Biak,ID,82.72,-0.91,122.88,
23,Kathu,TH,81.00,7.92,98.33,
28,Nortelandia,BR,88.56,-14.45,-56.80,
29,Tezu,IN,79.63,27.92,96.17,


### Create a Maximum Temperature Heatmap from a Hotel DataFrame

In [17]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [27]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [28]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]